In [16]:
from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

In [17]:
import numpy as np
import pyprind
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import pprint
import copy
import wurst as w
import brightway2 as bw
import bw2analyzer as bwa
from brightway2 import *

from matplotlib import pyplot as plt
from pandas import DataFrame
from wurst import *
from wurst.searching import *
from wurst.transformations.activity import change_exchanges_by_constant_factor
from wurst.transformations.uncertainty import rescale_exchange
from bw2data.parameters import ActivityParameter, DatabaseParameter, ProjectParameter, Group
from wurst.IMAGE.io import *
from wurst.IMAGE import *
from wurst.ecoinvent.electricity_markets import *
from wurst.ecoinvent.filters import *
from wurst.transformations.geo import *

In [18]:
projects.set_current("DAC_LCA")

## 1. Calculation of environmental impact of 1 kwh electricity generation 
###### These results are calculated for SSP2-RCP 1.9 w/o DACCS and SSP2-RCP 1.9 w/ DACCS scenarios. The results are presented in Figure 3b

In [ ]:
kwh_elec_database_dict = collections.defaultdict(dict)


kwh_elec_database_dict['RCP19'] = {'2020': {'database' : 'ecoinvent_2020_RCP19', 'result': []},
                             '2030': {'database' : 'ecoinvent_2030_RCP19', 'result': []},
                             '2040': {'database' : 'ecoinvent_2040_RCP19', 'result': []},
                             '2050': {'database' : 'ecoinvent_2050_RCP19', 'result': []},
                             '2060': {'database' : 'ecoinvent_2060_RCP19', 'result': []},
                             '2070': {'database' : 'ecoinvent_2070_RCP19', 'result': []},
                             '2080': {'database' : 'ecoinvent_2080_RCP19', 'result': []},
                             '2090': {'database' : 'ecoinvent_2090_RCP19', 'result': []},
                             '2100': {'database' : 'ecoinvent_2100_RCP19', 'result': []}}

kwh_elec_database_dict['RCP19_DAC'] = {'2020': {'database' : 'ecoinvent_2020_RCP19_DAC', 'result': []},
                             '2030': {'database' : 'ecoinvent_2030_RCP19_DAC', 'result': []},
                             '2040': {'database' : 'ecoinvent_2040_RCP19_DAC', 'result': []},
                             '2050': {'database' : 'ecoinvent_2050_RCP19_DAC', 'result': []},
                             '2060': {'database' : 'ecoinvent_2060_RCP19_DAC', 'result': []},
                             '2070': {'database' : 'ecoinvent_2070_RCP19_DAC', 'result': []},
                             '2080': {'database' : 'ecoinvent_2080_RCP19_DAC', 'result': []},
                             '2090': {'database' : 'ecoinvent_2090_RCP19_DAC', 'result': []},
                             '2100': {'database' : 'ecoinvent_2100_RCP19_DAC', 'result': []}}

#kwh_elec_database_dict

In [88]:
scenario = 'RCP19'

for year in kwh_elec_database_dict[scenario].keys():
    database_name = kwh_elec_database_dict[scenario][year]['database']
    ei_cf_36_db = bw.Database(database_name)
    hv_elec_us = [act for act in ei_cf_36_db if 'market group for electricity, high voltage' in act['name'] and 'US' in act['location']][0]
    lca_result = []
    lca = bw.LCA({hv_elec_us: 1})
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        lca_result.append((method[2].title(), lca.score))
    kwh_elec_database_dict[scenario][year]['result'] = lca_result

In [89]:
scenario = 'RCP19_DAC'

for year in kwh_elec_database_dict[scenario].keys():
    database_name = kwh_elec_database_dict[scenario][year]['database']
    ei_cf_36_db = bw.Database(database_name)
    hv_elec_us = [act for act in ei_cf_36_db if 'market group for electricity, high voltage' in act['name'] and 'US' in act['location']][0]
    lca_result = []
    lca = bw.LCA({hv_elec_us: 1})
    lca.lci()
    for method in method_key:
        lca.switch_method(method)
        lca.lcia()
        lca_result.append((method[2].title(), lca.score))
    kwh_elec_database_dict[scenario][year]['result'] = lca_result

In [90]:
scenario = 'RCP19'
appended_data = []
for year in kwh_elec_database_dict[scenario].keys():
    new_table = pd.DataFrame.from_dict(kwh_elec_database_dict[scenario][year]['result'])
    new_table['year'] = year
    appended_data.append(new_table)
    
appended_data = pd.concat(appended_data)
appended_data.columns = ['Impact_categories', 'value', 'year']
appended_data_spread_RCP19 = appended_data.pivot(index='year',columns='Impact_categories',values='value').copy()

appended_data_spread_RCP19.columns = ["Freshwater eutrophication (kg P-Eq)", 
                                      "Freshwater ecotoxicity (kg 14-DCB)", 
                                      'Climate change (kg CO2-Eq)', 
                                      "Human toxicity (kg 14-DCB)", 
                                      "Metal depletion (kg Fe-Eq)", 
                                      "Terrestrial acidification (kg SO2-Eq)", 
                                      "Terrestrial ecotoxicity (kg 14-DCB)", 
                                      "Water depletion (m3 water)"]
appended_data_spread_RCP19['Climate_scenario'] = scenario
appended_data_spread_RCP19

,Freshwater eutrophication (kg P-Eq),Freshwater ecotoxicity (kg 14-DCB),Climate change (kg CO2-Eq),Human toxicity (kg 14-DCB),Metal depletion (kg Fe-Eq),Terrestrial acidification (kg SO2-Eq),Terrestrial ecotoxicity (kg 14-DCB),Water depletion (m3 water),Climate_scenario
year,,,,,,,,,
2020,0.000480,0.010722,0.503807,0.317195,0.007159,0.001018,0.000029,0.002443,RCP19
2030,0.000088,0.006224,0.192496,0.114440,0.009017,0.000500,0.000039,0.001470,RCP19
2040,0.000065,0.007887,0.054514,0.071602,0.013384,0.000368,0.000041,0.001166,RCP19
2050,0.000055,0.009080,0.056860,0.048094,0.014812,0.000317,0.000044,0.000983,RCP19
2060,0.000042,0.009042,0.054517,0.031914,0.015364,0.000320,0.000045,0.000958,RCP19
2070,0.000034,0.009232,0.055471,0.024531,0.016283,0.000297,0.000049,0.000933,RCP19
2080,0.000026,0.009353,0.050714,0.024168,0.016577,0.000251,0.000051,0.000915,RCP19
2090,0.000024,0.010070,0.045723,0.025153,0.017524,0.000241,0.000053,0.000769,RCP19
2100,0.000023,0.010861,0.044670,0.026661,0.018356,0.000240,0.000053,0.000625,RCP19


In [91]:
scenario = 'RCP19_DAC'
appended_data = []
for year in kwh_elec_database_dict[scenario].keys():
    new_table = pd.DataFrame.from_dict(kwh_elec_database_dict[scenario][year]['result'])
    new_table['year'] = year
    appended_data.append(new_table)
    
appended_data = pd.concat(appended_data)
appended_data.columns = ['Impact_categories', 'value', 'year']
appended_data_spread_RCP19_DAC = appended_data.pivot(index='year',columns='Impact_categories',values='value').copy()

appended_data_spread_RCP19_DAC.columns = ["Freshwater eutrophication (kg P-Eq)", 
                                          "Freshwater ecotoxicity (kg 14-DCB)", 
                                          'Climate change (kg CO2-Eq)', 
                                          "Human toxicity (kg 14-DCB)", 
                                          "Metal depletion (kg Fe-Eq)", 
                                          "Terrestrial acidification (kg SO2-Eq)", 
                                          "Terrestrial ecotoxicity (kg 14-DCB)", 
                                          "Water depletion (m3 water)"]
appended_data_spread_RCP19_DAC['Climate_scenario'] = scenario
appended_data_spread_RCP19_DAC

,Freshwater eutrophication (kg P-Eq),Freshwater ecotoxicity (kg 14-DCB),Climate change (kg CO2-Eq),Human toxicity (kg 14-DCB),Metal depletion (kg Fe-Eq),Terrestrial acidification (kg SO2-Eq),Terrestrial ecotoxicity (kg 14-DCB),Water depletion (m3 water),Climate_scenario
year,,,,,,,,,
2020,0.000480,0.010722,0.503807,0.317195,0.007159,0.001018,0.000029,0.002443,RCP19_DAC
2030,0.000088,0.006224,0.192496,0.114440,0.009017,0.000500,0.000039,0.001470,RCP19_DAC
2040,0.000066,0.007878,0.054791,0.071657,0.013354,0.000369,0.000041,0.001167,RCP19_DAC
2050,0.000057,0.009165,0.057140,0.049227,0.014873,0.000318,0.000044,0.000980,RCP19_DAC
2060,0.000045,0.009482,0.058375,0.029026,0.016356,0.000332,0.000049,0.000901,RCP19_DAC
2070,0.000037,0.009675,0.058951,0.021637,0.017152,0.000309,0.000052,0.000864,RCP19_DAC
2080,0.000028,0.009903,0.055795,0.018219,0.017676,0.000268,0.000054,0.000825,RCP19_DAC
2090,0.000024,0.010587,0.050263,0.021972,0.018318,0.000251,0.000055,0.000685,RCP19_DAC
2100,0.000024,0.011173,0.049229,0.025800,0.018892,0.000248,0.000056,0.000570,RCP19_DAC


In [26]:
kwh_elec_sum_table = pd.concat([appended_data_spread_RCP19, appended_data_spread_RCP19_DAC]).copy()
kwh_elec_sum_table.to_excel(r'C:\Users\YQIU\Box\DAC LCA-IAM\YQ_work\result\Electricity_environmental_impact\US_1kwh_elec_impact_year_summary_table.xlsx', index = True)
kwh_elec_sum_table

,Freshwater eutrophication (kg P-Eq),Freshwater ecotoxicity (kg 14-DCB),Climate change (kg CO2-Eq),Human toxicity (kg 14-DCB),Metal depletion (kg Fe-Eq),Natural land transformation (square meter),Terrestrial acidification (kg SO2-Eq),Terrestrial ecotoxicity (kg 14-DCB),Water depletion (m3 water),Climate_scenario
year,,,,,,,,,,
2020,0.000487,0.014387,0.511526,0.325953,0.009185,0.000032,0.001044,0.000031,0.002472,RCP19
2030,0.000091,0.009851,0.197504,0.121453,0.011058,0.000026,0.000521,0.000041,0.001490,RCP19
2040,0.000066,0.011092,0.054867,0.072179,0.014517,0.000028,0.000428,0.000042,0.001146,RCP19
2050,0.000055,0.012210,0.056306,0.048159,0.015773,0.000033,0.000370,0.000044,0.000957,RCP19
2060,0.000041,0.011980,0.052267,0.030409,0.015908,0.000034,0.000377,0.000044,0.000916,RCP19
2070,0.000033,0.011794,0.050650,0.020273,0.016041,0.000032,0.000367,0.000047,0.000868,RCP19
2080,0.000025,0.012048,0.047194,0.021815,0.016636,0.000030,0.000310,0.000048,0.000863,RCP19
2090,0.000023,0.012900,0.043317,0.024040,0.017873,0.000026,0.000289,0.000051,0.000728,RCP19
2100,0.000023,0.013760,0.042585,0.026474,0.018840,0.000023,0.000282,0.000051,0.000592,RCP19


## 2. Calculating the contribution (by generation technologies) of environmental impacts of 1 kwh electricity generation
###### These results are calculated for SSP2-Baseline, SSP2-RCP 1.9 w/o DACCS, and SSP2-RCP 1.9 w/ DACCS scenarios. The results are presented in SI-Figure 9

In [5]:
database_dict = collections.defaultdict(dict)
#Baseline
database_dict['baseline'] = {'2020': {'database' : 'ecoinvent_2020_BAU', 'result': {}},
                             '2030': {'database' : 'ecoinvent_2030_BAU', 'result': {}},
                             '2040': {'database' : 'ecoinvent_2040_BAU', 'result': {}},
                             '2050': {'database' : 'ecoinvent_2050_BAU', 'result': {}},
                             '2060': {'database' : 'ecoinvent_2060_BAU', 'result': {}},
                             '2070': {'database' : 'ecoinvent_2070_BAU', 'result': {}},
                             '2080': {'database' : 'ecoinvent_2080_BAU', 'result': {}},
                             '2090': {'database' : 'ecoinvent_2090_BAU', 'result': {}},
                             '2100': {'database' : 'ecoinvent_2100_BAU', 'result': {}}}

In [94]:
elec_impact_RCP19_noDAC_dict = collections.defaultdict(dict)
#Baseline
elec_impact_RCP19_noDAC_dict['RCP19_noDAC'] = {'2020': {'database' : 'ecoinvent_2020_RCP19', 'result': {}},
                             '2030': {'database' : 'ecoinvent_2030_RCP19', 'result': {}},
                             '2040': {'database' : 'ecoinvent_2040_RCP19', 'result': {}},
                             '2050': {'database' : 'ecoinvent_2050_RCP19', 'result': {}},
                             '2060': {'database' : 'ecoinvent_2060_RCP19', 'result': {}},
                             '2070': {'database' : 'ecoinvent_2070_RCP19', 'result': {}},
                             '2080': {'database' : 'ecoinvent_2080_RCP19', 'result': {}},
                             '2090': {'database' : 'ecoinvent_2090_RCP19', 'result': {}},
                             '2100': {'database' : 'ecoinvent_2100_RCP19', 'result': {}}}

In [6]:
elec_impact_RCP19_dict = collections.defaultdict(dict)
#Baseline
elec_impact_RCP19_dict['RCP19'] = {'2020': {'database' : 'ecoinvent_2020_RCP19_DAC', 'result': {}},
                             '2030': {'database' : 'ecoinvent_2030_RCP19_DAC', 'result': {}},
                             '2040': {'database' : 'ecoinvent_2040_RCP19_DAC', 'result': {}},
                             '2050': {'database' : 'ecoinvent_2050_RCP19_DAC', 'result': {}},
                             '2060': {'database' : 'ecoinvent_2060_RCP19_DAC', 'result': {}},
                             '2070': {'database' : 'ecoinvent_2070_RCP19_DAC', 'result': {}},
                             '2080': {'database' : 'ecoinvent_2080_RCP19_DAC', 'result': {}},
                             '2090': {'database' : 'ecoinvent_2090_RCP19_DAC', 'result': {}},
                             '2100': {'database' : 'ecoinvent_2100_RCP19_DAC', 'result': {}}}

In [85]:
method_key = [
    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'climate change' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'ozone depletion' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'ionising radiation' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'photochemical oxidant formation' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'particulate matter formation' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'terrestrial acidification' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'freshwater ecotoxicity' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'marine ecotoxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'terrestrial ecotoxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'human toxicity' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'freshwater eutrophication' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'marine eutrophication' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'fossil depletion' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'metal depletion' in str(m)][0],
              [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'water depletion' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'agricultural land occupation' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'natural land transformation' in str(m)][0],
          #    [m for m in bw.methods if 'ReCiPe Midpoint (H) (obsolete)' in str(m) and 'urban land occupation' in str(m)][0]
             ]

### 2.1. BAU scenario

In [9]:
## BAU database
scenario = 'baseline'

for year in database_dict[scenario].keys():
    database_name = database_dict[scenario][year]['database']
    ei_cf_36_db = bw.Database(database_name)
    electricity_result = {}
    Electricity_US_hv = [act for act in ei_cf_36_db if 'market group for electricity, high voltage' == act['name'] and 'US' == act['location']][0]
    
    for hv in list(Electricity_US_hv.exchanges())[1:]:
        ## Remove the first exchange, which is the production
        grid_loc = hv['location']
        electricity_result[grid_loc] = {}
        hv_elec_us = [act for act in ei_cf_36_db if hv['name'] == act['name'] and hv['location'] == act['location']][0]
        
        for exc in list(hv_elec_us.exchanges()):
            if exc['type'] == 'production' or exc['type'] == 'biosphere' :
                continue
            activity = [act for act in ei_cf_36_db if exc['name'] in act['name'] and exc['location'] in act['location']][0]
            fun_unit = hv['amount']*exc['amount']
            electricity_result[grid_loc][activity['name'] + ', ' + activity['location']] = []
            lca = bw.LCA({activity: fun_unit})
            lca.lci()
            for method in method_key:
                lca.switch_method(method)
                lca.lcia()
                electricity_result[grid_loc][activity['name'] + ', ' + activity['location']].append((method[2].title(), lca.score))
    
    database_dict[scenario][year]['result'] = electricity_result.copy()

In [10]:
elec_impact_result_dict = collections.defaultdict(dict)
#Baseline
elec_impact_result_dict['baseline'] = {'2020': {},
                                       '2030': {},
                                       '2040': {},
                                       '2050': {},
                                       '2060': {},
                                       '2070': {},
                                       '2080': {},
                                       '2090': {},
                                       '2100': {}}

In [11]:
scenario = 'baseline'
year_range = ['2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2100']

electricity_impact_table = pd.DataFrame(index=['Climate change (kg CO2-Eq)',
                                               #'Ozone depletion (kg CFC-11)', 
                                               #'Ionising radiation (kg U235-Eq)', 
                                               #'Photochemical oxidant formation (kg NMVOC)', 
                                               #'Particulate matter formation (kg PM10-Eq)',
                                               'Terrestrial acidification (kg SO2-Eq)', 
                                               'Freshwater ecotoxicity (kg 14-DCB)',
                                               #'Marine ecotoxicity (kg 14-DCB)', 
                                               'Terrestrial ecotoxicity (kg 14-DCB)', 
                                               'Human toxicity (kg 14-DCB)',
                                               'Freshwater eutrophication (kg P-Eq)',
                                               #'Marine eutrophication (kg N-Eq)', 
                                               #'Fossil depletion (kg oil-Eq)',
                                               'Metal depletion (kg Fe-Eq)', 
                                               'Water depletion (m3 water)',
                                               #'Agricultural land occupation (square meter-year)',
                                               'Natural land transformation (square meter)', 
                                               #'Urban land occupation (square meter-year)'
                                              ])

for year in year_range:
    US_electricity_impact_dict = {}
    electricity_result = database_dict[scenario][year]['result']
    for region_key in list(electricity_result.keys()):
        US_electricity_impact_dict[region_key] = electricity_impact_table.copy()
        for process in electricity_result[region_key].keys():
            result_value = list(dict(electricity_result[region_key][process]).values())
            US_electricity_impact_dict[region_key][process] = result_value
    
    Merged_electrcity_impact_result = pd.concat(US_electricity_impact_dict.values(), axis=1).groupby(level=0, axis=1).sum()

    rename_list = list(Merged_electrcity_impact_result).copy()
    
    for i in range(0, len(list(Merged_electrcity_impact_result))):
        if 'electricity production, lignite' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite'
        elif 'electricity production, hard coal' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal'
        elif 'Electricity, at power plant/hard coal, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/hard coal, IGCC, no CCS/2025'
        elif 'Electricity, at power plant/lignite, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/lignite, IGCC, no CCS/2025'
        elif 'electricity production, natural gas, combined cycle power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, combined cycle power plant'
        elif 'electricity production, natural gas, conventional power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, conventional power plant'
        elif 'electricity production, nuclear, boiling water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, boiling water reactor'
        elif 'electricity production, nuclear, pressure water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, pressure water reactor'  
        elif 'electricity production, oil' in rename_list[i]:
            rename_list[i] = 'electricity production, oil'
        elif 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical'
        elif 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical'
        elif 'heat and power co-generation, oil' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, oil'
        elif 'heat and power co-generation, biogas, gas engine' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, biogas, gas engine'
        elif 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014'
        elif 'heat and power co-generation, hard coal' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, hard coal'
        elif 'heat and power co-generation, lignite' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, lignite'
        elif 'electricity production, wave, 20kw/meter' in rename_list[i]:
            rename_list[i] = 'electricity production, wave, 20kw/meter'
        elif 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted'
        elif 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si'
        elif 'electricity production, solar tower power plant, 20 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar tower power plant, 20 MW'
        elif 'electricity production, solar thermal parabolic trough, 50 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar thermal parabolic trough, 50 MW'
        elif 'electricity production, wind, 1-3MW turbine, offshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, offshore'
        elif 'electricity production, wind, 1-3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, onshore'
        elif 'electricity production, wind, <1MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, <1MW turbine, onshore'
        elif 'electricity production, wind, >3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, >3MW turbine, onshore'
        elif 'electricity production, wood, future' in rename_list[i]:
            rename_list[i] = 'electricity production, wood, future'
        elif 'Electricity, at BIGCC power plant 450MW, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at BIGCC power plant 450MW, no CCS/2025'
        elif 'electricity production, deep geothermal' in rename_list[i]:
            rename_list[i] = 'electricity production, deep geothermal'
        elif 'electricity production, hydro, reservoir, alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, alpine region'
        elif 'electricity production, hydro, reservoir, non-alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, non-alpine region'
        elif 'electricity production, hydro, run-of-river' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, run-of-river'
        elif 'market for electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for electricity, high voltage'
        elif 'market for transmission network, electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for transmission network, electricity, high voltage'
        elif 'market for transmission network, long-distance' in rename_list[i]:
            rename_list[i] = 'market for transmission network, long-distance'
    
    Renamed_merged_electrcity_impact_result = Merged_electrcity_impact_result.copy()
    Renamed_merged_electrcity_impact_result.columns = rename_list
    
    
    elec_impact_result_dict['baseline'][year] = Renamed_merged_electrcity_impact_result.groupby(level=0, axis=1).sum().copy()

In [12]:
Elec_impact_summary_table = pd.concat(elec_impact_result_dict['baseline']).fillna(0).copy()
Elec_impact_summary_table = Elec_impact_summary_table.reset_index()
Elec_impact_summary_table = pd.melt(Elec_impact_summary_table, id_vars=["level_0", "level_1"], var_name="tech", value_name="value").copy()
Elec_impact_output_table = Elec_impact_summary_table.rename(columns = {'level_0': 'year', 'level_1': 'impact_category'}).copy()
Elec_impact_output_table

,year,impact_category,tech,value
0,2020,Climate change (kg CO2-Eq),"electricity production, deep geothermal",2.934253e-04
1,2020,Terrestrial acidification (kg SO2-Eq),"electricity production, deep geothermal",9.099024e-07
2,2020,Freshwater ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",9.299727e-06
3,2020,Terrestrial ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",1.245474e-07
4,2020,Human toxicity (kg 14-DCB),"electricity production, deep geothermal",2.777007e-04
...,...,...,...,...
2668,2100,Human toxicity (kg 14-DCB),"Electricity, at BIGCC power plant 450MW, no CC...",-4.336376e-04
2669,2100,Freshwater eutrophication (kg P-Eq),"Electricity, at BIGCC power plant 450MW, no CC...",4.284757e-07
2670,2100,Metal depletion (kg Fe-Eq),"Electricity, at BIGCC power plant 450MW, no CC...",1.797823e-04
2671,2100,Water depletion (m3 water),"Electricity, at BIGCC power plant 450MW, no CC...",4.589956e-06


In [13]:
elec_impact_path = r'C:\Users\yqiu\Box\DAC LCA-IAM\YQ_work\result\Electricity_environmental_impact\US_hv_SSP2_Baseline.xlsx'

writer = pd.ExcelWriter(elec_impact_path, engine = 'xlsxwriter')
Elec_impact_output_table.to_excel(writer, index=False)

writer.save()
writer.close()

C:\Users\yqiu\Anaconda3\envs\DAC_LCA_project\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 2.2. RCP1.9 no DAC scenario

In [98]:
## RCP1.9_no DAC scenario
scenario = 'RCP19_noDAC'

for year in elec_impact_RCP19_noDAC_dict[scenario].keys():
    database_name = elec_impact_RCP19_noDAC_dict[scenario][year]['database']
    ei_cf_36_db = bw.Database(database_name)
    electricity_result = {}
    Electricity_US_hv = [act for act in ei_cf_36_db if 'market group for electricity, high voltage' == act['name'] and 'US' == act['location']][0]
    
    for hv in list(Electricity_US_hv.exchanges())[1:]:
        ## Remove the first exchange, which is the production
        grid_loc = hv['location']
        electricity_result[grid_loc] = {}
        hv_elec_us = [act for act in ei_cf_36_db if hv['name'] == act['name'] and hv['location'] == act['location']][0]
        
        for exc in list(hv_elec_us.exchanges()):
            if exc['type'] == 'production' or exc['type'] == 'biosphere' :
                continue
            activity = [act for act in ei_cf_36_db if exc['name'] in act['name'] and exc['location'] in act['location']][0]
            fun_unit = hv['amount']*exc['amount']
            new_exchange = activity['name'] + ', ' + activity['location'] + ', ' + str(fun_unit)
            
            lca_result = []
            #electricity_result[grid_loc][activity['name'] + ', ' + activity['location']] = []
            lca = bw.LCA({activity: fun_unit})
            lca.lci()
            for method in method_key:
                lca.switch_method(method)
                lca.lcia()
                lca_result.append((method[2].title(), lca.score))
                
            electricity_result[grid_loc].update({new_exchange:lca_result})
    
    elec_impact_RCP19_noDAC_dict[scenario][year]['result'] = electricity_result.copy()

In [99]:
elec_impact_RCP19_noDAC_result_dict = collections.defaultdict(dict)
#Baseline
elec_impact_RCP19_noDAC_result_dict['RCP19_noDAC'] = {'2020': {},
                                       '2030': {},
                                       '2040': {},
                                       '2050': {},
                                       '2060': {},
                                       '2070': {},
                                       '2080': {},
                                       '2090': {},
                                       '2100': {}}

In [100]:
scenario = 'RCP19_noDAC'
year_range = ['2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2100']

electricity_impact_table = pd.DataFrame(index=['Climate change (kg CO2-Eq)',
                                               #'Ozone depletion (kg CFC-11)', 
                                               #'Ionising radiation (kg U235-Eq)', 
                                               #'Photochemical oxidant formation (kg NMVOC)', 
                                               #'Particulate matter formation (kg PM10-Eq)',
                                               'Terrestrial acidification (kg SO2-Eq)', 
                                               'Freshwater ecotoxicity (kg 14-DCB)',
                                               #'Marine ecotoxicity (kg 14-DCB)', 
                                               'Terrestrial ecotoxicity (kg 14-DCB)', 
                                               'Human toxicity (kg 14-DCB)',
                                               'Freshwater eutrophication (kg P-Eq)',
                                               #'Marine eutrophication (kg N-Eq)', 
                                               #'Fossil depletion (kg oil-Eq)',
                                               'Metal depletion (kg Fe-Eq)', 
                                               'Water depletion (m3 water)',
                                               #'Agricultural land occupation (square meter-year)',
                                               #'Natural land transformation (square meter)', 
                                               #'Urban land occupation (square meter-year)'
                                              ])

for year in year_range:
    US_electricity_impact_dict = {}
    electricity_result = elec_impact_RCP19_noDAC_dict[scenario][year]['result']
    for region_key in list(electricity_result.keys()):
        US_electricity_impact_dict[region_key] = electricity_impact_table.copy()
        for process in electricity_result[region_key].keys():
            result_value = list(dict(electricity_result[region_key][process]).values())
            US_electricity_impact_dict[region_key][process] = result_value
    
    Merged_electrcity_impact_result = pd.concat(US_electricity_impact_dict.values(), axis=1).groupby(level=0, axis=1).sum()

    rename_list = list(Merged_electrcity_impact_result).copy()
    
    for i in range(0, len(list(Merged_electrcity_impact_result))):
        if 'electricity production, lignite' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite'
        elif 'electricity production, hard coal' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal'
        elif 'Electricity, at power plant/hard coal, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/hard coal, IGCC, no CCS/2025'
        elif 'Electricity, at power plant/lignite, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/lignite, IGCC, no CCS/2025'
        elif 'electricity production, natural gas, combined cycle power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, combined cycle power plant'
        elif 'electricity production, natural gas, conventional power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, conventional power plant'
        elif 'electricity production, nuclear, boiling water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, boiling water reactor'
        elif 'electricity production, nuclear, pressure water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, pressure water reactor'  
        elif 'electricity production, oil' in rename_list[i]:
            rename_list[i] = 'electricity production, oil'
        elif 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical'
        elif 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical'
        elif 'heat and power co-generation, oil' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, oil'
        elif 'heat and power co-generation, biogas, gas engine' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, biogas, gas engine'
        elif 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014'
        elif 'heat and power co-generation, hard coal' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, hard coal'
        elif 'heat and power co-generation, lignite' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, lignite'
        elif 'electricity production, wave' in rename_list[i]:
            rename_list[i] = 'electricity production, wave'
        elif 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted'
        elif 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si'
        elif 'electricity production, solar tower power plant, 20 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar tower power plant, 20 MW'
        elif 'electricity production, solar thermal parabolic trough, 50 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar thermal parabolic trough, 50 MW'
        elif 'electricity production, wind, 1-3MW turbine, offshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, offshore'
        elif 'electricity production, wind, 1-3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, onshore'
        elif 'electricity production, wind, <1MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, <1MW turbine, onshore'
        elif 'electricity production, wind, >3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, >3MW turbine, onshore'
        elif 'electricity production, wood, future' in rename_list[i]:
            rename_list[i] = 'electricity production, wood, future'
        elif 'Electricity, at BIGCC power plant 450MW, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at BIGCC power plant 450MW, no CCS/2025'
        elif 'electricity production, deep geothermal' in rename_list[i]:
            rename_list[i] = 'electricity production, deep geothermal'
        elif 'electricity production, hydro, reservoir, alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, alpine region'
        elif 'electricity production, hydro, reservoir, non-alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, non-alpine region'
        elif 'electricity production, hydro, run-of-river' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, run-of-river'
        elif 'Electricity, at power plant/hard coal, oxy, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'
        elif 'Electricity, at power plant/hard coal, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'    
        elif 'Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'             
        elif 'Electricity, at power plant/lignite, oxy, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'
        elif 'Electricity, at power plant/lignite, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'    
        elif 'Electricity, at power plant/lignite, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'  
        elif 'Electricity, at power plant/natural gas, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas with CCS'
        elif 'Electricity, at power plant/natural gas, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas with CCS'    
        elif 'Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'     
        elif 'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'
        elif 'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'            
        elif 'market for electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for electricity, high voltage'
        elif 'market for transmission network, electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for transmission network, electricity, high voltage'
        elif 'market for transmission network, long-distance' in rename_list[i]:
            rename_list[i] = 'market for transmission network, long-distance'
    
    Renamed_merged_electrcity_impact_result = Merged_electrcity_impact_result.copy()
    Renamed_merged_electrcity_impact_result.columns = rename_list
    
    
    elec_impact_RCP19_noDAC_result_dict['RCP19_noDAC'][year] = Renamed_merged_electrcity_impact_result.groupby(level=0, axis=1).sum().copy()

In [101]:
Elec_RCP19_noDAC_impact_summary_table = pd.concat(elec_impact_RCP19_noDAC_result_dict['RCP19_noDAC']).fillna(0).copy()
Elec_RCP19_noDAC_impact_summary_table = Elec_RCP19_noDAC_impact_summary_table.reset_index()
Elec_RCP19_noDAC_impact_summary_table = pd.melt(Elec_RCP19_noDAC_impact_summary_table, id_vars=["level_0", "level_1"], var_name="tech", value_name="value").copy()
Elec_RCP19_noDAC_impact_output_table = Elec_RCP19_noDAC_impact_summary_table.rename(columns = {'level_0': 'year', 'level_1': 'impact_category'}).copy()
Elec_RCP19_noDAC_impact_output_table

,year,impact_category,tech,value
0,2020,Climate change (kg CO2-Eq),"electricity production, deep geothermal",2.934253e-04
1,2020,Terrestrial acidification (kg SO2-Eq),"electricity production, deep geothermal",9.099024e-07
2,2020,Freshwater ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",9.299727e-06
3,2020,Terrestrial ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",1.245474e-07
4,2020,Human toxicity (kg 14-DCB),"electricity production, deep geothermal",2.777007e-04
...,...,...,...,...
2443,2100,Terrestrial ecotoxicity (kg 14-DCB),"electricity production, natural gas with CCS",4.205764e-07
2444,2100,Human toxicity (kg 14-DCB),"electricity production, natural gas with CCS",6.313167e-04
2445,2100,Freshwater eutrophication (kg P-Eq),"electricity production, natural gas with CCS",4.446976e-07
2446,2100,Metal depletion (kg Fe-Eq),"electricity production, natural gas with CCS",4.566396e-04


In [102]:
elec_impact_path = r'C:\Users\yqiu\Box\DAC LCA-IAM\YQ_work\result\Electricity_environmental_impact\US_hv_SSP2_RCP19_noDAC.xlsx'


writer = pd.ExcelWriter(elec_impact_path, engine = 'xlsxwriter')
Elec_RCP19_noDAC_impact_output_table.to_excel(writer, index=False)

writer.save()
writer.close()

C:\Users\yqiu\Anaconda3\envs\DAC_LCA_project\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


### 2.3. RCP1.9 with DAC scenario

In [14]:
## RCP1.9_with DAC scenario
scenario = 'RCP19'

for year in elec_impact_RCP19_dict[scenario].keys():
    database_name = elec_impact_RCP19_dict[scenario][year]['database']
    ei_cf_36_db = bw.Database(database_name)
    electricity_result = {}
    Electricity_US_hv = [act for act in ei_cf_36_db if 'market group for electricity, high voltage' == act['name'] and 'US' == act['location']][0]
    
    for hv in list(Electricity_US_hv.exchanges())[1:]:
        ## Remove the first exchange, which is the production
        grid_loc = hv['location']
        electricity_result[grid_loc] = {}
        hv_elec_us = [act for act in ei_cf_36_db if hv['name'] == act['name'] and hv['location'] == act['location']][0]
        
        for exc in list(hv_elec_us.exchanges()):
            if exc['type'] == 'production' or exc['type'] == 'biosphere' :
                continue
            activity = [act for act in ei_cf_36_db if exc['name'] in act['name'] and exc['location'] in act['location']][0]
            fun_unit = hv['amount']*exc['amount']
            new_exchange = activity['name'] + ', ' + activity['location'] + ', ' + str(fun_unit)
            
            lca_result = []
            #electricity_result[grid_loc][activity['name'] + ', ' + activity['location']] = []
            lca = bw.LCA({activity: fun_unit})
            lca.lci()
            for method in method_key:
                lca.switch_method(method)
                lca.lcia()
                lca_result.append((method[2].title(), lca.score))
                
            electricity_result[grid_loc].update({new_exchange:lca_result})
    
    elec_impact_RCP19_dict[scenario][year]['result'] = electricity_result.copy()

In [15]:
elec_impact_RCP19_result_dict = collections.defaultdict(dict)
#Baseline
elec_impact_RCP19_result_dict['RCP19'] = {'2020': {},
                                       '2030': {},
                                       '2040': {},
                                       '2050': {},
                                       '2060': {},
                                       '2070': {},
                                       '2080': {},
                                       '2090': {},
                                       '2100': {}}

In [16]:
scenario = 'RCP19'
year_range = ['2020', '2030', '2040', '2050', '2060', '2070', '2080', '2090', '2100']

electricity_impact_table = pd.DataFrame(index=['Climate change (kg CO2-Eq)',
                                               #'Ozone depletion (kg CFC-11)', 
                                               #'Ionising radiation (kg U235-Eq)', 
                                               #'Photochemical oxidant formation (kg NMVOC)', 
                                               #'Particulate matter formation (kg PM10-Eq)',
                                               'Terrestrial acidification (kg SO2-Eq)', 
                                               'Freshwater ecotoxicity (kg 14-DCB)',
                                               #'Marine ecotoxicity (kg 14-DCB)', 
                                               'Terrestrial ecotoxicity (kg 14-DCB)', 
                                               'Human toxicity (kg 14-DCB)',
                                               'Freshwater eutrophication (kg P-Eq)',
                                               #'Marine eutrophication (kg N-Eq)', 
                                               #'Fossil depletion (kg oil-Eq)',
                                               'Metal depletion (kg Fe-Eq)', 
                                               'Water depletion (m3 water)',
                                               #'Agricultural land occupation (square meter-year)',
                                               'Natural land transformation (square meter)', 
                                               #'Urban land occupation (square meter-year)'
                                              ])

for year in year_range:
    US_electricity_impact_dict = {}
    electricity_result = elec_impact_RCP19_dict[scenario][year]['result']
    for region_key in list(electricity_result.keys()):
        US_electricity_impact_dict[region_key] = electricity_impact_table.copy()
        for process in electricity_result[region_key].keys():
            result_value = list(dict(electricity_result[region_key][process]).values())
            US_electricity_impact_dict[region_key][process] = result_value
    
    Merged_electrcity_impact_result = pd.concat(US_electricity_impact_dict.values(), axis=1).groupby(level=0, axis=1).sum()

    rename_list = list(Merged_electrcity_impact_result).copy()
    
    for i in range(0, len(list(Merged_electrcity_impact_result))):
        if 'electricity production, lignite' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite'
        elif 'electricity production, hard coal' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal'
        elif 'Electricity, at power plant/hard coal, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/hard coal, IGCC, no CCS/2025'
        elif 'Electricity, at power plant/lignite, IGCC, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at power plant/lignite, IGCC, no CCS/2025'
        elif 'electricity production, natural gas, combined cycle power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, combined cycle power plant'
        elif 'electricity production, natural gas, conventional power plant' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas, conventional power plant'
        elif 'electricity production, nuclear, boiling water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, boiling water reactor'
        elif 'electricity production, nuclear, pressure water reactor' in rename_list[i]:
            rename_list[i] = 'electricity production, nuclear, pressure water reactor'  
        elif 'electricity production, oil' in rename_list[i]:
            rename_list[i] = 'electricity production, oil'
        elif 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical'
        elif 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical'
        elif 'heat and power co-generation, oil' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, oil'
        elif 'heat and power co-generation, biogas, gas engine' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, biogas, gas engine'
        elif 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, wood chips, 6667 kW, state-of-the-art 2014'
        elif 'heat and power co-generation, hard coal' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, hard coal'
        elif 'heat and power co-generation, lignite' in rename_list[i]:
            rename_list[i] = 'heat and power co-generation, lignite'
        elif 'electricity production, wave' in rename_list[i]:
            rename_list[i] = 'electricity production, wave'
        elif 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted'
        elif 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si' in rename_list[i]:
            rename_list[i] = 'electricity production, photovoltaic, 570kWp open ground installation, multi-Si'
        elif 'electricity production, solar tower power plant, 20 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar tower power plant, 20 MW'
        elif 'electricity production, solar thermal parabolic trough, 50 MW' in rename_list[i]:
            rename_list[i] = 'electricity production, solar thermal parabolic trough, 50 MW'
        elif 'electricity production, wind, 1-3MW turbine, offshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, offshore'
        elif 'electricity production, wind, 1-3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, 1-3MW turbine, onshore'
        elif 'electricity production, wind, <1MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, <1MW turbine, onshore'
        elif 'electricity production, wind, >3MW turbine, onshore' in rename_list[i]:
            rename_list[i] = 'electricity production, wind, >3MW turbine, onshore'
        elif 'electricity production, wood, future' in rename_list[i]:
            rename_list[i] = 'electricity production, wood, future'
        elif 'Electricity, at BIGCC power plant 450MW, no CCS/2025' in rename_list[i]:
            rename_list[i] = 'Electricity, at BIGCC power plant 450MW, no CCS/2025'
        elif 'electricity production, deep geothermal' in rename_list[i]:
            rename_list[i] = 'electricity production, deep geothermal'
        elif 'electricity production, hydro, reservoir, alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, alpine region'
        elif 'electricity production, hydro, reservoir, non-alpine region' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, reservoir, non-alpine region'
        elif 'electricity production, hydro, run-of-river' in rename_list[i]:
            rename_list[i] = 'electricity production, hydro, run-of-river'
        elif 'Electricity, at power plant/hard coal, oxy, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'
        elif 'Electricity, at power plant/hard coal, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'    
        elif 'Electricity, at power plant/hard coal, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, hard coal with CCS'             
        elif 'Electricity, at power plant/lignite, oxy, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'
        elif 'Electricity, at power plant/lignite, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'    
        elif 'Electricity, at power plant/lignite, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, lignite with CCS'  
        elif 'Electricity, at power plant/natural gas, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas with CCS'
        elif 'Electricity, at power plant/natural gas, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, natural gas with CCS'    
        elif 'Electricity, from CC plant, 100% SNG, truck 25km, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'     
        elif 'Electricity, at wood burning power plant 20 MW, truck 25km, post, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'
        elif 'Electricity, at BIGCC power plant 450MW, pre, pipeline 200km, storage 1000m/2025, US' in rename_list[i]:
            rename_list[i] = 'electricity production, biomass with CCS'            
        elif 'market for electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for electricity, high voltage'
        elif 'market for transmission network, electricity, high voltage' in rename_list[i]:
            rename_list[i] = 'market for transmission network, electricity, high voltage'
        elif 'market for transmission network, long-distance' in rename_list[i]:
            rename_list[i] = 'market for transmission network, long-distance'
    
    Renamed_merged_electrcity_impact_result = Merged_electrcity_impact_result.copy()
    Renamed_merged_electrcity_impact_result.columns = rename_list
    
    
    elec_impact_RCP19_result_dict['RCP19'][year] = Renamed_merged_electrcity_impact_result.groupby(level=0, axis=1).sum().copy()

In [17]:
Elec_RCP19_impact_summary_table = pd.concat(elec_impact_RCP19_result_dict['RCP19']).fillna(0).copy()
Elec_RCP19_impact_summary_table = Elec_RCP19_impact_summary_table.reset_index()
Elec_RCP19_impact_summary_table = pd.melt(Elec_RCP19_impact_summary_table, id_vars=["level_0", "level_1"], var_name="tech", value_name="value").copy()
Elec_RCP19_impact_output_table = Elec_RCP19_impact_summary_table.rename(columns = {'level_0': 'year', 'level_1': 'impact_category'}).copy()
Elec_RCP19_impact_output_table

,year,impact_category,tech,value
0,2020,Climate change (kg CO2-Eq),"electricity production, deep geothermal",2.934253e-04
1,2020,Terrestrial acidification (kg SO2-Eq),"electricity production, deep geothermal",9.099024e-07
2,2020,Freshwater ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",9.299727e-06
3,2020,Terrestrial ecotoxicity (kg 14-DCB),"electricity production, deep geothermal",1.245474e-07
4,2020,Human toxicity (kg 14-DCB),"electricity production, deep geothermal",2.777007e-04
...,...,...,...,...
2749,2100,Human toxicity (kg 14-DCB),"electricity production, natural gas with CCS",6.225756e-04
2750,2100,Freshwater eutrophication (kg P-Eq),"electricity production, natural gas with CCS",4.457672e-07
2751,2100,Metal depletion (kg Fe-Eq),"electricity production, natural gas with CCS",4.606706e-04
2752,2100,Water depletion (m3 water),"electricity production, natural gas with CCS",2.537371e-04


In [18]:
elec_impact_path = r'C:\Users\yqiu\Box\DAC LCA-IAM\YQ_work\result\Electricity_environmental_impact\US_hv_SSP2_RCP19_DAC.xlsx'


writer = pd.ExcelWriter(elec_impact_path, engine = 'xlsxwriter')
Elec_RCP19_impact_output_table.to_excel(writer, index=False)

writer.save()
writer.close()

C:\Users\yqiu\Anaconda3\envs\DAC_LCA_project\lib\site-packages\xlsxwriter\workbook.py:329: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")
